# Comparing Temperatures and Luminosities Between Analyses

This notebook measures global temperatures and luminosities for the samples we are using to verify our method, then compares them to the measurements from literature. While the focus of this work is the measurement of galaxy cluster masses, temperatures and luminosities play an important role in the construction of X-ray mass-observable scaling relations - thus they must also be shown to be consistent with previous work. 

## Import Statements

In [1]:
import pandas as pd
import numpy as np
from astropy.units import Quantity
from astropy.cosmology import LambdaCDM, WMAP9
import matplotlib.pyplot as plt

from xga.samples import ClusterSample
from xga.xspec import single_temp_apec

## Defining useful functions

In [2]:
# def find_lims(x_dat, y_dat, buffer=0.1):
#     lom = 1 - buffer
#     him 1 + buffer
    
#     x_vals = x_dat[:, 0]
#     y_vals = y_dat[:, 0]
    
#     if x_dat.shape[1] == 2:
#         x_lims = [lom*min(x_vals-x_dat[:, 1])]
        

## Setting up cosmology

Though three of the previous analyses use the same concordance cosmology, the XXL analysis of their bright cluster sample uses the WMAP9 results, and in case we wish to change anything for an individual sample later we define four separate cosmology objects to pass into our samples.

In [3]:
xcs_cosmo = LambdaCDM(70, 0.3, 0.7)
xxl_cosmo = WMAP9
locuss_cosmo = LambdaCDM(70, 0.3, 0.7)
lovisari_cosmo = LambdaCDM(70, 0.3, 0.7)

## Reading in Sample Files and Declaring XGA ClusterSamples

$\color{red}{\text{NEED TO MENTION/PUBLISH THE OBSIDS THAT WE EXCLUDE FROM USE DUE TO DATA PROBLEMS LIKE FLARING}}$

This subsection involves reading in the sample files of the four test samples (described in [the sample properties notebook](sample_properties.ipynb)), then setting up separate XGA ClusterSample instances (see [the documentation](https://xga.readthedocs.io/en/latest/notebooks/tutorials/sources_samples.html) for an introduction to XGA source and sample objects.

We impose an additional cleaning step on each sample, where we make sure that (for each XMM observation initially associated with a source) at least 70% of a cluster's $R_{500}$ falls on that observation - if this requirement is not met then the observation is excluded. These requirements are set with the `clean_obs=True`, `clean_obs_reg='r500'`, and `clean_obs_threshold=0.7` arguments when a ClusterSample instance is declared.

### SDSSRM-XCS Volume Limited

This is the recent SDSSRM-XCS sample. The temperatures and luminosities are measured by the XCS luminosity-temperature pipeline, and with this we demonstrate that XGA temperatures and luminosities are consistent with existing XCS results.

In order to achieve maximum consistency, we use the XAPA coordinates as the central position for spectrum generation (turning off the XGA peak finder with `use_peak=False`). We have also made sure to use the same cosmology.

In [4]:
xcs3p = pd.read_csv("sample_files/xcs3p_sdssrm_vol_lim_temperr_25%_clusters.csv")

In [5]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = xcs3p['xapa_ra'].values
dec = xcs3p['xapa_dec'].values
z = xcs3p['z'].values
# Not using the IAU names in XCS_NAME column, its easier for me to use the name based on redMaPPer ID
n = xcs3p['name'].values
# In arcminutes, ClusterSample declaration will convert to kpc using the provided cosmology
r500 = Quantity(xcs3p['r500'].values, 'arcmin')
# Not likely to use richness in this notebook, but I'm putting it in the sample object anyway
r = xcs3p['richness'].values
r_err = xcs3p['richness_err'].values

# Declaring the actual ClusterSample instance for the XCS sample
xcs_srcs = ClusterSample(ra, dec, z, n, r500=r500, richness=r, richness_err=r_err, cosmology=xcs_cosmo, 
                         load_fits=True, use_peak=False, clean_obs=True, clean_obs_reg='r500', 
                         clean_obs_threshold=0.7)

Setting up Galaxy Clusters:   6%|▉             | 10/157 [00:08<02:05,  1.17it/s]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-11331 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:   8%|█             | 12/157 [00:12<03:10,  1.32s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-28849 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:   8%|█▏            | 13/157 [00:28<13:50,  5.77s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-31144 has not be

Setting up Galaxy Clusters:  22%|███           | 35/157 [03:11<18:19,  9.01s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-120 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  29%|████          | 46/157 [03:35<02:05,  1.13s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-325 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  31%|████▎         | 48/157 [03:38<02:21,  1.30s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-5077 has not been de

Setting up Galaxy Clusters:  64%|████████▎    | 101/157 [09:53<01:11,  1.27s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:109: UserWarning: There are 1 alternative matches for observation 0109080301, associated with source SDSSXCS-14715
  warnings.warn("There are {0} alternative matches for observation {1}, associated with "
/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-14715 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  66%|████████▌    | 104/157 [09:57<01:00,  1.13s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-457 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n

Generating products of type(s) expmap: 100%|████| 53/53 [00:04<00:00, 11.34it/s]


### XXL-100-GC

This is the sample of the brightest clusters in the XXL survey. It contains temperature and luminosity measurements, and will be a useful external comparison for XGA results, though the shallow XXL data may prove a challenge.

In [6]:
xxlgc100 = pd.read_csv("sample_files/xxl_gc100.csv")

# Limit the comparison to clusters with a flag of 0 - meaning it is in the main sample of 100 clusters
xxlgc100 = xxlgc100[xxlgc100['Flag'] == 0]

In [7]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = xxlgc100['ra'].values
dec = xxlgc100['dec'].values
z = xxlgc100['z'].values
n = xxlgc100['name'].values
r500 = Quantity(xxlgc100['r500MT'].values, 'Mpc')

# Declaring the actual ClusterSample instance for the XXL sample
# This is the only sample whose original analysis used the WMAP9 cosmology
xxl_srcs = ClusterSample(ra, dec, z, n, r500=r500, cosmology=xxl_cosmo, load_fits=True, use_peak=False, 
                         clean_obs=True, clean_obs_reg='r500', clean_obs_threshold=0.7)

Setting up Galaxy Clusters:   0%|                       | 0/100 [00:00<?, ?it/s]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-94 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:   1%|▏              | 1/100 [00:07<12:03,  7.31s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-96 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:   2%|▎              | 2/100 [00:18<15:53,  9.73s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:109: UserWarning: There are 1 alternative matches fo

Setting up Galaxy Clusters:  18%|██▌           | 18/100 [02:11<08:09,  5.96s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-99 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  19%|██▋           | 19/100 [02:17<08:01,  5.95s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-97 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  20%|██▊           | 20/100 [02:25<08:49,  6.62s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-110 has not been detected in

/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-25 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  34%|████▊         | 34/100 [07:07<30:53, 28.08s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-41 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  35%|████▉         | 35/100 [07:54<36:39, 33.84s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-50 has not been detected in all region files, so generating and fitting products with the 'region' reg_type 

Setting up Galaxy Clusters:  48%|██████▋       | 48/100 [12:06<13:07, 15.14s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-87 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  49%|██████▊       | 49/100 [12:14<11:11, 13.16s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-91 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  50%|███████       | 50/100 [12:28<11:06, 13.33s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-105 has not been detected in

Setting up Galaxy Clusters:  65%|█████████     | 65/100 [13:26<02:04,  3.55s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-523 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  66%|█████████▏    | 66/100 [13:30<02:00,  3.55s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-503 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  67%|█████████▍    | 67/100 [13:37<02:30,  4.57s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:109: UserWarning: There are 1 alternative matches 

/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-546 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  80%|███████████▏  | 80/100 [14:36<01:13,  3.69s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-512 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  81%|███████████▎  | 81/100 [14:42<01:24,  4.46s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-520 has not been detected in all region files, so generating and fitting products with the 'region' reg_ty

Setting up Galaxy Clusters:  98%|█████████████▋| 98/100 [16:09<00:12,  6.12s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-510 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  99%|█████████████▊| 99/100 [16:13<00:05,  5.35s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: XLSSC-511 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Generating products of type(s) expmap: 100%|████| 88/88 [00:09<00:00,  8.96it/s]


### LoCuSS High-$L_{\rm{X}}$

The LoCuSS High-$L_{\rm{X}}$ sample was selected from ROSAT for its high luminosity clusters, and will again be a useful comparison as testing against various different analyses is beneficial in establishing the veracity of our new measurements.

In [8]:
locuss = pd.read_csv("sample_files/locuss_highlx_clusters.csv", dtype={'chandra_id': str, 'xmm_obsid': str})

In [9]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = locuss['ra'].values
dec = locuss['dec'].values
z = locuss['z'].values
n = locuss['name'].values
r500 = Quantity(locuss['r500'].values, 'kpc')
r2500 = Quantity(locuss['r2500'].values, 'kpc')


# Declaring the actual ClusterSample instance for the LoCuSS sample
locuss_srcs = ClusterSample(ra, dec, z, n, r500=r500, r2500=r2500, cosmology=locuss_cosmo, load_fits=True, 
                            use_peak=False, clean_obs=True, clean_obs_reg='r500', clean_obs_threshold=0.7)

Declaring BaseSource Sample:  36%|█████         | 18/50 [00:15<00:31,  1.02it/s]/home/dt237/code/PycharmProjects/XGA/xga/samples/base.py:91: UserWarning: Source ZwCl0949.6+5207 does not appear to have any XMM data, and will not be included in the sample.
  warn("Source {n} does not appear to have any XMM data, and will not be included in the "
Declaring BaseSource Sample:  46%|██████▍       | 23/50 [00:21<00:32,  1.19s/it]/home/dt237/code/PycharmProjects/XGA/xga/samples/base.py:91: UserWarning: Source Abell1423 does not appear to have any XMM data, and will not be included in the sample.
  warn("Source {n} does not appear to have any XMM data, and will not be included in the "
Declaring BaseSource Sample:  54%|███████▌      | 27/50 [00:24<00:20,  1.14it/s]/home/dt237/code/PycharmProjects/XGA/xga/samples/base.py:91: UserWarning: Source Abell1682 does not appear to have any XMM data, and will not be included in the sample.
  warn("Source {n} does not appear to have any XMM data, and will

### Planck Selected with XMM follow-up

The Lovisari et al. sample was selected from Planck-eSZ, and will again be a useful comparison as testing against various different analyses is beneficial in establishing the veracity of our new measurements.

In [10]:
lovisari = pd.read_csv("sample_files/lovisari_planck_clusters.csv")

In [11]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = lovisari['planck_ra'].values
dec = lovisari['planck_dec'].values
z = lovisari['z'].values
n = lovisari['name'].values
r500 = Quantity(lovisari['r500'].values, 'kpc')

# Declaring the actual ClusterSample instance for the Lovisari sample
lovisari_srcs = ClusterSample(ra, dec, z, n, r500=r500, cosmology=lovisari_cosmo, load_fits=True, use_peak=False, 
                            clean_obs=True, clean_obs_reg='r500', clean_obs_threshold=0.7)

Setting up Galaxy Clusters:   3%|▌              | 4/120 [00:05<02:47,  1.45s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: G006.78+30.46 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:   8%|█▏            | 10/120 [00:27<04:52,  2.66s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/extended.py:226: UserWarning: A PSF sized extended source has been detected in 0202080201 and is very close to the user supplied coordinates of G042.82+56.61. It will not be excluded from analysis due to the possibility of a mis-identified cool core
  warnings.warn("A PSF sized extended source has been detected in {o} and is very close to the "
/home/dt237/code/PycharmProjects/XGA/xga/sources/extended.py:222: UserWarning: A point source has been detected in 

Setting up Galaxy Clusters:  52%|███████▎      | 63/120 [02:26<04:38,  4.88s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/extended.py:222: UserWarning: A point source has been detected in 0723161001 and is very close to the user supplied coordinates of G228.49+53.12. It will not be excluded from analysis due to the possibility of a mis-identified cool core
  warnings.warn("A point source has been detected in {o} and is very close to the user supplied "
Setting up Galaxy Clusters:  54%|███████▌      | 65/120 [02:28<02:46,  3.02s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: G229.94+15.29 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  62%|████████▋     | 74/120 [02:44<00:58,  1.27s/it]/home/dt237/code/PycharmProjects/XGA/xga/source

## Running $T_{\rm{X}}$ and $L_{\rm{X}}$ Measurements

The XGA XSPEC functions that we use here all automatically call the XGA SAS interface, so the necessary spectra are generated before the fits begin. As the different samples measure properties within different spatial regions, all the function calls differ slightly.

The results of the fits are stored within the indivual source objects that make up each sample.

###  SDSSRM-XCS

For our comparisons here we wish to measure the temperature ($T_{\rm{X}}$) and luminosity ($L_\rm{X}$; both in the 0.5-2.0 keV and bolometric/0.01-100.0 keV energy bands) within $R_{\rm{500}}$. We fit a `constant*tbabs*apec` model; $\color{red}{\text{with the choices for absorption (`tbabs`) and plasma emission (`tbabs`) consistent with the XCS analysis, though the addition of a multiplicative constant to manage differences in sensitivity is different from the original analysis.}}$

In [ ]:
single_temp_apec(xcs_srcs, xcs_srcs.r500)

Generating products of type(s) spectrum:  79%|▊| 552/699 [8:39:06<2:56:48, 72.16

### XXL-100-GC

The XXL temperatures and soft-band $L_{\rm{X}}$ are measured within 300 kpc, which we replicate by setting the outer radius to an astropy quantity of 300 kpc.

In [ ]:
single_temp_apec(xxl_srcs, Quantity(300, 'kpc'))

### LoCuSS

The LoCuSS temperatures and luminosities are slightly different, and have been measured with the core of the X-ray emission removed (spectra are generated in the 0.15-1 $R_{500}$ region). This approach has been shown to reduce the scatter in mass with $L_{\rm{X}}$. We can replicate it by setting an inner radius when we call `single_temp_apec`.

In [ ]:
single_temp_apec(locuss_srcs, locuss_srcs.r500, inner_radius=0.15*locuss_srcs.r500)

### Lovisari 

The Lovisari sample's has both core-excised and core-included temperatures and luminosities, so we shall also measure both. The Lovisari analysis used a different energy range for the soft-band luminosity measurements than the other samples (0.1-2.4 keV). We can replicate this by changing the default luminosity energy bands used by `single_temp_apec`, so we measure core excluded and included luminosities in the 0.5-2.0 keV, 0.01-100.0 keV (bolometric), and 0.1-2.4 keV bands.

In [ ]:
lovisari_lum_en = Quantity([[0.5, 2.0], [0.01, 100.0], [0.1, 2.4]], 'keV')

single_temp_apec(lovisari_srcs, lovisari_srcs.r500, lum_en=lovisari_lum_en)
single_temp_apec(lovisari_srcs, lovisari_srcs.r500, inner_radius=0.15*lovisari_srcs.r500, 
                 lum_en=lovisari_lum_en)

## Retrieving $T_{\rm{X}}$ and $L_{\rm{X}}$ measurements from the samples

We must extract and judge the quality of the temperature and luminosity measurements that we have made for each of the samples, then later on we will be able to directly compare them. At the same time we make sure that the results from literature are formatted in such a way that we can easily compare them.

### SDSSRM-XCS

In [ ]:
stop

In [ ]:
sdss_tx_all = xcs_srcs.Tx(xcs_srcs.r500, quality_checks=False).value
sdss_tx = xcs_srcs.Tx(xcs_srcs.r500, quality_checks=True)

sdss_lxbol_all = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=False, lo_en=Quantity(0.01, 'keV'), 
                             hi_en=Quantity(100.0, 'keV'))
sdss_lxbol = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=True, lo_en=Quantity(0.01, 'keV'), 
                         hi_en=Quantity(100.0, 'keV'))

sdss_lx_all = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=False, lo_en=Quantity(0.5, 'keV'), 
                             hi_en=Quantity(2.0, 'keV'))
sdss_lx = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=True, lo_en=Quantity(0.5, 'keV'), 
                         hi_en=Quantity(2.0, 'keV'))

In [ ]:
xcs3p_tx = Quantity(xcs3p[['Tx', 'Tx-', 'Tx+']].values, 'keV')
xcs3p_lx52 = Quantity(xcs3p[['Lx52', 'Lx52-', 'Lx52+']].values, '1e+44 erg/s')
xcs3p_lxbol = Quantity(xcs3p[['Lx', 'Lx-', 'Lx+']].values, '1e+44 erg/s')

### XXL-100-GC

In [ ]:
xxl_tx_all = xxl_srcs.Tx(Quantity(300, 'kpc'), quality_checks=False)
xxl_tx = xxl_srcs.Tx(Quantity(300, 'kpc'), quality_checks=True)

### LoCuSS

In [ ]:
locuss_txce_all = locuss_srcs.Tx(locuss_srcs.r500, inner_radius=0.15*locuss_srcs.r500, quality_checks=False)
locuss_txce = locuss_srcs.Tx(locuss_srcs.r500, inner_radius=0.15*locuss_srcs.r500, quality_checks=True)

### Lovisari

In [ ]:
lovisari_txce_all = lovisari_srcs.Tx(lovisari_srcs.r500, inner_radius=0.15*lovisari_srcs.r500, 
                                     quality_checks=False)
lovisari_txce = lovisari_srcs.Tx(lovisari_srcs.r500, inner_radius=0.15*lovisari_srcs.r500, 
                                 quality_checks=True)

lovisari_tx_all = lovisari_srcs.Tx(lovisari_srcs.r500, quality_checks=False)
lovisari_tx = lovisari_srcs.Tx(lovisari_srcs.r500, quality_checks=True)

lovisari_lxce_all = lovisari_srcs.Lx(lovisari_srcs.r500, inner_radius=0.15*lovisari_srcs.r500, 
                                     quality_checks=False, lo_en=Quantity(0.1, 'keV'), hi_en=Quantity(2.4, 'keV'))
lovisari_lxce = lovisari_srcs.Lx(lovisari_srcs.r500, inner_radius=0.15*lovisari_srcs.r500, quality_checks=True, 
                               lo_en=Quantity(0.1, 'keV'), hi_en=Quantity(2.4, 'keV'))

lovisari_lx_all = lovisari_srcs.Lx(lovisari_srcs.r500, quality_checks=False, lo_en=Quantity(0.1, 'keV'), 
                                 hi_en=Quantity(2.4, 'keV'))
lovisari_lx = lovisari_srcs.Lx(lovisari_srcs.r500, quality_checks=True, lo_en=Quantity(0.1, 'keV'), 
                             hi_en=Quantity(2.4, 'keV'))

lovisari_lxbolce_all = lovisari_srcs.Lx(lovisari_srcs.r500, inner_radius=0.15*lovisari_srcs.r500, 
                                        quality_checks=False, lo_en=Quantity(0.01, 'keV'), 
                                        hi_en=Quantity(100.0, 'keV'))
lovisari_lxbolce = lovisari_srcs.Lx(lovisari_srcs.r500, inner_radius=0.15*lovisari_srcs.r500, quality_checks=True, 
                                  lo_en=Quantity(0.01, 'keV'), hi_en=Quantity(100.0, 'keV'))

lovisari_lxbol_all = lovisari_srcs.Lx(lovisari_srcs.r500, quality_checks=False, lo_en=Quantity(0.01, 'keV'), 
                                      hi_en=Quantity(100.0, 'keV'))
lovisari_lxbol = lovisari_srcs.Lx(lovisari_srcs.r500, quality_checks=True, lo_en=Quantity(0.01, 'keV'), 
                                  hi_en=Quantity(100.0, 'keV'))

## Direct comparisons between original and XGA measurements

This is the the point of this notebook, making direct comparisons of like for like (or as near as we can achieve) measurements from literature to measurements made using XGA. In each case we plot simple one-to-one comparisons, with a one-to-one dashed line in red to give a reference.

### SDSSRM-XCS

We directly compare temperatures and luminosities (soft band and bolimetric) between the XCS3P pipeline and XGA.

#### $T_{\rm{X}, 500}$, $L^{\rm{0.5-2.0}}_{\rm{X}, 500}$, and $L^{\rm{bol}}_{\rm{X}, 500}$ 

In [ ]:
fig, ax_arr = plt.subplots(ncols=3, figsize=(17, 6))

# Iterating through the array of axes objects, setting up the ticks
for ax in ax_arr:
    ax.minorticks_on()
    ax.tick_params(which='both', top=True, right=True, direction='in')
    
plt.sca(ax_arr[0])


plt.errorbar(xcs3p_tx[:, 0].value, sdss_tx[:, 0].value, xerr=[xcs3p_tx[:, 1].value, xcs3p_tx[:, 2].value], 
             fmt="kx", capsize=2, label="")

plt.tight_layout()
plt.show()

### XXL-100-GC

### LoCuSS

### Lovisari

### Combined